# Studi Kasus Beer Production

## PROFILE KELOMPOK


NPM KETUA : 200411100153

NAMA KETUA : FIKRI AINUN NAJIB

NPM ANGGOTA : 

NAMA ANGGOTA : 

GITHUB : https://github.com/LALA09-erha/kolaborasipro

DATASET : https://www.kaggle.com/datasets/shenba/time-series-datasets?select=monthly-beer-production-in-austr.csv

MODEL : KNN REGRESI , GAUSSIAN REGRESI

KELAS PSD C

Inisialisasi Libary

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from numpy import array
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel


Pengambilan data

In [21]:
data = pd.read_csv("monthly-beer-production-in-austr.csv")
data = data ['Monthly beer production']


Split Data

In [22]:
data_training,data_test = train_test_split(data, train_size=0.8, test_size=0.2, shuffle=False, random_state=42)

In [23]:
data_training_new = pd.DataFrame(data_training)
data_test_new = pd.DataFrame(data_test)

In [24]:
sc = MinMaxScaler()
train_scaled = sc.fit_transform(data_training_new)
test_scaled = sc.transform(data_test_new)

In [25]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
  X, y = list(), list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
  
  return array(X), array(y)
# define univariate time series

In [26]:
n_steps = range(1,4)
acc_fix = {}

for n in n_steps:
    # transform to a supervised learning problem
    X_train, y_train = split_sequence(train_scaled, n) 
    X_test, y_test = split_sequence(test_scaled, n) 

    # membuat kolom otomatis
    colom = []
    for c in range(n):
        if(c==0):
            colom.insert(0,"Xt")
        else:
            colom.insert(0,f"Xt-{c}")

    # # Masukkan ke dataframe hasil dari split  squens
    # UNTUK MEMISAHKAN X TRAIN, YTRAIN dan TEST
    newX_train = pd.DataFrame(np.reshape(X_train, (len(X_train) , n)),columns=colom)
    newY_train = pd.DataFrame(y_train, columns=['Output'])
    new_data_train = pd.concat([newX_train,newY_train],axis=1)

    newX_test = pd.DataFrame(np.reshape(X_test, (len(X_test) , n)),columns=colom)
    newY_test = pd.DataFrame(y_test, columns=['Output'])
    new_data_test = pd.concat([newX_test,newY_test],axis=1)


    # EKSEKUSI KE model

    # predict kkn
    k_nei = range(1,30)
    acc = []
    for k in k_nei:
        knn = KNeighborsRegressor(n_neighbors = k)
        knn.fit(newX_train,newY_train)
        y_pred = knn.predict(newX_test)
        y_pred_shape = y_pred.reshape(-1,1)
        y_pred_inverse = sc.inverse_transform(y_pred_shape)
        newY_test_shape = y_test.reshape(-1,1)
        newY_test_inverse = sc.inverse_transform(newY_test_shape)
        mape = mean_absolute_percentage_error(y_pred_inverse, newY_test_inverse)
        acc.append(mape)
    
    # masukkan ke acc-fix 
    acc_fix[f"n-{n} kkn"] = [min(acc) , acc.index(min(acc))]
        
    # predict gaussian regressor
    kernel = DotProduct() + WhiteKernel()
    gpr = GaussianProcessRegressor(kernel=kernel,random_state=0).fit(newX_train, newY_train)
    y_pred = gpr.predict(newX_test, return_std=True)[0]
    y_pred_shape = y_pred.reshape(-1,1)
    y_pred_inverse = sc.inverse_transform(y_pred_shape)
    newY_test_shape = y_test.reshape(-1,1)
    newY_test_inverse = sc.inverse_transform(newY_test_shape)
    mape = mean_absolute_percentage_error(y_pred_inverse, newY_test_inverse)

    acc_fix[f"n-{n} gpr"] = [mape]
print(acc_fix)
 




{'n-1 kkn': [0.10181916745221799, 28], 'n-1 gpr': [0.1068229173306441], 'n-2 kkn': [0.10028274680900791, 16], 'n-2 gpr': [0.10906140141958477], 'n-3 kkn': [0.09585157865328107, 6], 'n-3 gpr': [0.10781311991232205]}
